In [2]:
gene=Array.new #Creating an empty array for saving each line of the document
File.foreach("documents/ArabidopsisSubNetwork_GeneList.txt"){ |line|  gene <<line } #Saving each line of the document
                                                      # in the array. Source: https://www.rubyguides.com/2015/05/working-with-files-ruby/
gene_p=Array.new   #Creating an empty array for saving procesed name of genes (it has metacharacter \n)
gene.each do |line| 
  gene_p << line.gsub("\n",'') #Eliminating metacharacter \n in each line. 
end   #The difference between gene and gene_p array is that in the second one we can access to each value gene_p[1]
     # and the return has not metacharacter \n

["AT4g27030\n", "AT5g54270\n", "AT1g21400\n", "AT5g19120\n", "AT2g13360\n", "AT4g05180\n", "AT1g22690\n", "AT2g45170\n", "AT4g09650\n", "AT5g55620\n", "AT1g31330\n", "AT1g80440\n", "AT3g28270\n", "AT4g12800\n", "AT5g04140\n", "AT5g64040\n", "AT1g29660\n", "AT1g15820\n", "AT1g64500\n", "AT1g03130\n", "AT2g20670\n", "AT4g17090\n", "AT1g23740\n", "AT2g46340\n", "AT4g17460\n", "AT1g07010\n", "AT1g32220\n", "AT2g03750\n", "AT3g47470\n", "AT4g12830\n", "AT5g05690\n", "AT5g64410\n", "AT2g21330\n", "AT1g15980\n", "AT2g01890\n", "AT1g12250\n", "AT2g21210\n", "AT4g28660\n", "AT1g52230\n", "AT3g18890\n", "AT4g22890\n", "AT1g11850\n", "AT1g37130\n", "AT2g04039\n", "AT3g48420\n", "AT4g19170\n", "AT5g07020\n", "AT5g65010\n", "AT3g01500\n", "AT1g25230\n", "AT2g34430\n", "AT1g18060\n", "AT2g30600\n", "AT4g33010\n", "AT1g65490\n", "AT3g21670\n", "AT4g27260\n", "AT1g12780\n", "AT1g55480\n", "AT2g17880\n", "AT3g49160\n", "AT4g21210\n", "AT5g14740\n", "AT4g37980\n", "AT1g29395\n", "AT2g39730\n", "AT1g2002

In [3]:
gene[1]

"AT5g54270\n"

In [4]:
gene_p[2]

"AT1g21400"

Haria falta un bucle para extraer el uniprot kb de cada gen:

El siguiente bucle devuelve todas las intereacciones que tienen todas las proteinas de los genes de arriba (hria falta limpiar y quitar uniprotkb: y quedarnos con el numero)

Con el  objetivo de buscar las interacciones que tiene cada uno de los genes por separado.

Mas adelante se pueden unir estos dos bucles en uno y hacerlo todo de un tiron

In [4]:
require 'rest-client'   # This access to the page of each gene 

#We have to search in other data base the interactions
intactstring=Array.new
gene_p.each do |gene_id|
  address = "http://www.ebi.ac.uk/Tools/webservices/psicquic/intact/webservices/current/search/interactor/#{gene_id}?format=tab25"
  response = RestClient::Request.execute(method: :get, url: address)
  record = response.body.gsub("\t","\n") #Changes tab for new line
  record_su = record.split(/\n/)  #split the string to substrings based on new line pattern: https://www.geeksforgeeks.org/ruby-string-split-method-with-examples/
  
  record_su.each do |line|
  intactstring <<  line.match(/.*intact.*/)
  end
end

Interrupt: 

In [5]:
puts uniprot_ID

NameError: undefined local variable or method `uniprot_ID' for #<Object:0x0000556079817598 @prompt={:PROMPT_I=>"3.0.0 :%03n > ", :PROMPT_S=>"3.0.0 :%03n%l> ", :PROMPT_C=>"3.0.0 :%03n > ", :PROMPT_N=>"3.0.0 :%03n?> ", :RETURN=>" => %s \n", :AUTO_INDENT=>true}>

Esta linea de abajo es para ir probando con un unico gen

In [73]:
require 'rest-client'
gene_id="At4g18960"
#unless gene_id.match(/AT\dg\ddddd/)
#  abort
#end
address = "http://www.ebi.ac.uk/Tools/webservices/psicquic/intact/webservices/current/search/interactor/#{gene_id}?format=tab25"
response = RestClient::Request.execute(method: :get, url: address)
record = response.body
record_su= record.split(/\n/) #split the string to substrings based on new line pattern: https://www.geeksforgeeks.org/ruby-string-split-method-with-examples/
                              #separating by each interaction
interactor1=[]
interactor2=[]

record_su.each do |line|
  temp=line.split(/\t/)
  #sometimes one interactor is lose, eliminating these
  unless temp[3].match(/.*AT\dg\ddddd:.*/) and temp[2].match(/.*AT\dg\ddddd:.*/)
    next("HOLA")
  else
    temp2=temp[2].gsub("|","\n")
    temp_2=temp2.split(/\n/)
    interactor1.append(temp_2)
    temp3=temp[3].gsub("|","\n")
    temp_3=temp3.split(/\n/)
    interactor2.append(temp_3)
    interactor1_s=[]
    interactor1.each do |line|
      line.each do |key|
        interactor1_s.append(key) if key.match(/ensemblplants:/)
      end
    end
    interactor2_s=[]
    interactor2.each do |line|
      line.each do |key|
        interactor2_s.append(key) if key.match(/ensemblplants:/)
      end
    end
    
    #Tengo interactor 1 e interactor 2 en una lista. Aquí me he quedao, no puec mes
    interactions=[]
    interactor1_s.each do |line|
      temp_1 = line.split(":")[1]
        interactor2_s.each do |line2|
          temp_2 = line2.split(":")[1]
          interactions.append(temp_1,temp_2)
      end
      
  end
  end
end


#A veces la proteina va primera, a veces va segunda
#A veces la misma interaccion se repite pero con las proteinas cambiadas de orden
#La data base devuelve TODAS las interacciones de esa proteína dada

["uniprotkb:P17839\tuniprotkb:P17839\tintact:EBI-592365|uniprotkb:O82732|ensemblplants:AT4G18960.1\tintact:EBI-592365|uniprotkb:O82732|ensemblplants:AT4G18960.1\tpsi-mi:ag_arath(display_long)|uniprotkb:F13C5.130(orf name)|uniprotkb:AG(gene name)|psi-mi:AG(display_short)|uniprotkb:At4g18960(locus name)\tpsi-mi:ag_arath(display_long)|uniprotkb:F13C5.130(orf name)|uniprotkb:AG(gene name)|psi-mi:AG(display_short)|uniprotkb:At4g18960(locus name)\tpsi-mi:\"MI:0018\"(two hybrid)\tFavaro et al. (2003)\tpubmed:14555696\ttaxid:3702(arath)|taxid:3702(\"Arabidopsis thaliana (Mouse-ear cress)\")\ttaxid:3702(arath)|taxid:3702(\"Arabidopsis thaliana (Mouse-ear cress)\")\tpsi-mi:\"MI:0915\"(physical association)\tpsi-mi:\"MI:0469\"(IntAct)\tintact:EBI-592383\tintact-miscore:0.37", "uniprotkb:P17839\tuniprotkb:O22456\tintact:EBI-592365|uniprotkb:O82732|ensemblplants:AT4G18960.1\tintact:EBI-592020|uniprotkb:Q8GWQ4|uniprotkb:Q8L9R5|ensemblplants:AT1G24260.2\tpsi-mi:ag_arath(display_long)|uniprotkb:F13C5.

In [70]:
puts "#{interactions}"

NameError: undefined local variable or method `interactions' for #<Object:0x000055a0ad0245d8 @prompt={:PROMPT_I=>"3.0.0 :%03n > ", :PROMPT_S=>"3.0.0 :%03n%l> ", :PROMPT_C=>"3.0.0 :%03n > ", :PROMPT_N=>"3.0.0 :%03n?> ", :RETURN=>" => %s \n", :AUTO_INDENT=>true}>
Did you mean?  interactor2
               interactor1

In [7]:
i1=interactor_1_s.split(",").to_s

"[\"[[\\\"intact:EBI-592365\\\"\", \" \\\"uniprotkb:O82732\\\"\", \" \\\"ensemblplants:AT4G18960.1\\\"]\", \" [\\\"intact:EBI-592365\\\"\", \" \\\"uniprotkb:O82732\\\"\", \" \\\"ensemblplants:AT4G18960.1\\\"]\", \" [\\\"intact:EBI-592343\\\"\", \" \\\"uniprotkb:Q8VZ36\\\"\", \" \\\"ensemblplants:AT4G09960.1\\\"]\", \" [\\\"intact:EBI-592365\\\"\", \" \\\"uniprotkb:O82732\\\"\", \" \\\"ensemblplants:AT4G18960.1\\\"]\", \" [\\\"intact:EBI-592343\\\"\", \" \\\"uniprotkb:Q8VZ36\\\"\", \" \\\"ensemblplants:AT4G09960.1\\\"]\", \" [\\\"intact:EBI-592304\\\"\", \" \\\"ensemblplants:AT3G58780.1\\\"]\", \" [\\\"intact:EBI-621903\\\"\", \" \\\"ensemblplants:AT3G02310.1\\\"\", \" \\\"uniprotkb:A0A178V680\\\"]\", \" [\\\"intact:EBI-621912\\\"\", \" \\\"ensemblplants:AT5G60910.1\\\"]\", \" [\\\"intact:EBI-621921\\\"\", \" \\\"uniprotkb:Q9M2F0\\\"\", \" \\\"ensemblplants:AT3G61120.1\\\"]\", \" [\\\"intact:EBI-621930\\\"\", \" \\\"uniprotkb:Q9FPP0\\\"\", \" \\\"uniprotkb:Q7XYY8\\\"\", \" \\\"uniprotkb

In [8]:
i1.match

ArgumentError: wrong number of arguments (given 0, expected 1..2)

In [17]:
interactor1[0]["ensemblplants"]

TypeError: no implicit conversion of String into Integer

www.ebi.ac.uk/intact/ws/interactor  BASE URL


it needs that we convert genes id into Uniprot
It provides UNiprot gene A interacts with UNiprot gene B

In [ ]:
#.match(/___/i) ignora el case sensitive

In [12]:
require 'rest-client'
gene_id="At4g18960"
#unless gene_id.match(/AT\dg\ddddd/)
#  abort
#end
address = "http://www.ebi.ac.uk/Tools/webservices/psicquic/intact/webservices/current/search/interactor/#{gene_id}?format=tab25"
response = RestClient::Request.execute(method: :get, url: address)
record = response.body.gsub("\t","\n") #Changes tab for new line
record_su = record.split(/\n/)  #split the string to substrings based on new line pattern: https://www.geeksforgeeks.org/ruby-string-split-method-with-examples/
puts record_su[

uniprotkb:P17839


In [170]:
require 'rest-client'
gene_id="At4g18960"
#unless gene_id.match(/AT\dg\ddddd/)
#  abort
#end
address = "http://www.ebi.ac.uk/Tools/webservices/psicquic/intact/webservices/current/search/interactor/#{gene_id}?format=tab25"
response = RestClient::Request.execute(method: :get, url: address)
record = response.body

puts response.body

uniprotkb:P17839	uniprotkb:P17839	intact:EBI-592365|uniprotkb:O82732|ensemblplants:AT4G18960.1	intact:EBI-592365|uniprotkb:O82732|ensemblplants:AT4G18960.1	psi-mi:ag_arath(display_long)|uniprotkb:F13C5.130(orf name)|uniprotkb:AG(gene name)|psi-mi:AG(display_short)|uniprotkb:At4g18960(locus name)	psi-mi:ag_arath(display_long)|uniprotkb:F13C5.130(orf name)|uniprotkb:AG(gene name)|psi-mi:AG(display_short)|uniprotkb:At4g18960(locus name)	psi-mi:"MI:0018"(two hybrid)	Favaro et al. (2003)	pubmed:14555696	taxid:3702(arath)|taxid:3702("Arabidopsis thaliana (Mouse-ear cress)")	taxid:3702(arath)|taxid:3702("Arabidopsis thaliana (Mouse-ear cress)")	psi-mi:"MI:0915"(physical association)	psi-mi:"MI:0469"(IntAct)	intact:EBI-592383	intact-miscore:0.37
uniprotkb:P17839	uniprotkb:O22456	intact:EBI-592365|uniprotkb:O82732|ensemblplants:AT4G18960.1	intact:EBI-592020|uniprotkb:Q8GWQ4|uniprotkb:Q8L9R5|ensemblplants:AT1G24260.2	psi-mi:ag_arath(display_long)|uniprotkb:F13C5.130(orf name)|uniprotkb:AG(gene n